In [ ]:
import cv2
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np

# 1. 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ResNet50GaugeNet(nn.Module):
    def __init__(self, num_keypoints=4):
        super(ResNet50GaugeNet, self).__init__()
        # [변경 부분]: 백본을 ResNet-50으로 교체 (Bottleneck 구조 사용)
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        # 특징 추출부 (마지막 AvgPool 및 FC 제외)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2]) 
        
        # [변경 부분]: ResNet-50의 최종 출력 채널은 2048입니다.
        # 2048 -> 512 -> 256 -> 128 -> num_keypoints 단계로 복원
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(2048, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, num_keypoints, kernel_size=1) 
        )
        
    def forward(self, x):
        x = self.backbone(x) # [Batch, 2048, H/32, W/32]
        x = self.deconv(x)   # [Batch, num_keypoints, H/4, W/4]
        return x

# 모델 초기화
model = ResNet50GaugeNet(num_keypoints=4).to(device).eval()

# 젯슨 나노에서 ResNet-50을 돌리기 위해 FP16(Half)은 "선택이 아닌 필수"입니다.
if device.type == 'cuda':
    model = model.half()

# 2. GStreamer 파이프라인 (안정성을 위해 448x448 권장)
def gstreamer_pipeline(sensor_id=0, flip_method=2):
    return (
        f"nvarguscamerasrc sensor-id={sensor_id} ! "
        f"video/x-raw(memory:NVMM), width=1280, height=720, framerate=30/1 ! "
        f"nvvidconv left=280 right=1000 top=0 bottom=720 flip-method={flip_method} ! " 
        f"video/x-raw, width=448, height=448, format=BGRx ! "      
        f"videoconvert ! video/x-raw, format=BGR ! appsink"
    )

def run_resnet50_gauge():
    # 실행 전 sudo systemctl restart nvargus-daemon 잊지 마세요!
    cap = cv2.VideoCapture(gstreamer_pipeline(), cv2.CAP_GSTREAMER)
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            # 전처리 및 정규화 (ResNet-50은 정규화에 더 민감합니다)
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_normalized = img_rgb.astype(np.float32) / 255.0
            # ImageNet 정규화 값 적용
            img_normalized = (img_normalized - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
            
            tensor = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0).to(device)
            if device.type == 'cuda': tensor = tensor.half()

            # 추론
            with torch.no_grad():
                heatmaps = model(tensor)

            # 결과 시각화
            hm = heatmaps[0].cpu().float().numpy()
            points = []
            for i in range(hm.shape[0]):
                # 히트맵 노이즈 제거를 위한 블러링
                smoothed_hm = cv2.GaussianBlur(hm[i], (3, 3), 0)
                _, conf, _, max_loc = cv2.minMaxLoc(smoothed_hm)
                
                x = int(max_loc[0] * (448 / hm.shape[2]))
                y = int(max_loc[1] * (448 / hm.shape[1]))
                points.append((x, y))
                
                # 가시성을 위해 빨간색 점 표시
                cv2.circle(frame, (x, y), 6, (0, 0, 255), -1)

            # 바늘 연결 (중심점 0번 - 끝점 1번)
            if len(points) >= 2:
                cv2.line(frame, points[0], points[1], (0, 255, 0), 2)

            cv2.imshow("ResNet-50 High-Precision Gauge", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): break
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    run_resnet50_gauge()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/arkplus/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:09<00:00, 11.4MB/s]


GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3280 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3280 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: Running with following settings:
   Camera index = 0 
   Camera mode  = 4 
   Output Stream W = 1280 H = 7

[ WARN:0@480.122] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


GST_ARGUS: Cleaning up
CONSUMER: Done Success
GST_ARGUS: Done Success
